# 🚀 Хакатон: Задание 1. Тюнинг и Оценка LLM

Всем привет! Наша команда занимается созданием чат-бота на основе RAG (https://habr.com/ru/articles/779526/) для помощи студентам НИУ ВШЭ с ответами на их вопросы, касающихся университета.

Для корректной работы нашей модели нам каждый раз приходится передавать ей системный промпт. Ознакомиться с ним можете в файле system_prompt.yaml.

Каждый раз его передавать в модель не хочется, поэтому нам бы хотелось иметь модель, которая *уже будет натюнена следовать инструкциям*, прописанным в системном промпте без дополнительных указаний.

##**Условия и какие данные мы предоставляем**

> Условия


1.   Натюньте побольше легковесных моделей. Ниже есть код для некоторых разных вариантов, пробуйте другие, пробуйте те, что указаны в коде (про тюнинг, его виды и прочее можете почитать здесь и здесь.) [Просто прикольная презентация про ллмки, виды обучения, дообучения, бенчмарки](https://mellain.github.io/data/YNDX_Meetup_LLM_Train_Slides_2024.pdf)

2.   Отвалидируйте модель (подобрать параметры, посмотреть на то, что происходит с лоссом). Советуем использовать [wandb](https://wandb.ai/)

3. Отобразить в красивом, понятном, интепретируемом виде результаты☺️☺️☺️

P.S. не забудьте сохранить одну лучшую модель, по итогам мы прогоним ваши модели на скрытой выборке и определим победителей☠️


> Что мы предоставляем

##**Код, который может быть полезен :)**

###Все импорты

In [ ]:
%%capture
%pip install transformers datasets accelerate rouge-score nltk bitsandbytes peft

In [ ]:
import json
import math
from tqdm import tqdm
import os
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from datasets import load_dataset

### Загрузка данных для обучения

### Код для загрузки различных моделей (можете использовать любые, но не берите слишком тяжеловесные, на бесплатных гпу с колаба вы не успеете ее обучить :)))

### Класс для обучения

In [ ]:
class ModelTrainer:
    def __init__(self, model_name: str, task: str = "causal", quantization: bool = True):
        self.model_name = model_name
        self.task = task
        self.quantization = quantization
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model, self.tokenizer = self.load_model()

    def load_model(self):
        kwargs = {"load_in_4bit": True, "device_map": "auto"} if self.quantization else {"device_map": "auto"}

        if self.task == "causal":
            model = AutoModelForCausalLM.from_pretrained(self.model_name, **kwargs)
        elif self.task == "classification":
            model = AutoModelForSequenceClassification.from_pretrained(self.model_name, **kwargs)
        else:
            raise ValueError("Неподдерживаемая задача")

        tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        model.to(self.device)
        return model, tokenizer

    def load_data(self, data_path: str):
        dataset = load_dataset("json", data_files=data_path)
        return dataset.map(lambda examples: self.tokenizer(examples["question"] + " " + examples.get("context", ""),
                                                      truncation=True, padding="max_length", max_length=512),
                                                      batched=True)

    def train(self, train_data_path: str, output_dir: str = "./results", epochs: int = 3, batch_size: int = 2):
        os.makedirs(output_dir, exist_ok=True)

        tokenized_datasets = self.load_data(train_data_path)
        train_dataloader = torch.utils.data.DataLoader(tokenized_datasets["train"], batch_size=batch_size, shuffle=True)

        self.model.train()
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=5e-5)

        for epoch in range(epochs):
            progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)

            for batch in progress_bar:
                batch = {k: v.to(self.model.device) for k, v in batch.items()}
                optimizer.zero_grad()
                outputs = self.model(**batch)
                loss = outputs.loss
                loss.backward()
                optimizer.step()

                progress_bar.set_postfix(loss=loss.item())

        self.model.save_pretrained(output_dir)
        print(f"Модель {self.model_name} сохранена в {output_dir}.")

### Обучаем

In [ ]:
available_models = {
    "causal": ["ai-forever/saiga_mistral_7b", "mistralai/Mistral-7B-Instruct", "SberbankAI/FRED-T5-1.7B"],
    "classification": ["distilbert-base-uncased", "bert-tiny"]
} #гуглите любые, это просто примерус

selected_model = "ai-forever/saiga_mistral_7b"
trainer = ModelTrainer(selected_model)
trainer.train("train_data.json")

###Код для подсчета метрик вашей модельки :)

In [ ]:
def compute_rouge(reference, hypothesis):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    return scores[0]["rouge-l"]["f"]

def compute_bleu(reference, hypothesis):
    return sentence_bleu([reference.split()], hypothesis.split())

def compute_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
    return math.exp(loss.item())

In [ ]:
test_data = json.load(open("generated_answers.json", "r", encoding="utf-8"))

rouge_scores, bleu_scores, perplexity_scores = [], [], []

for item in test_data:
    ref = item["gold_answer"]
    hyp = item["generated_answer"]

    rouge_scores.append(compute_rouge(ref, hyp))
    bleu_scores.append(compute_bleu(ref, hyp))
    perplexity_scores.append(compute_perplexity(model, tokenizer, hyp))

print(f"Средний ROUGE-L: {sum(rouge_scores) / len(rouge_scores)}")
print(f"Средний BLEU: {sum(bleu_scores) / len(bleu_scores)}")
print(f"Средний Perplexity: {sum(perplexity_scores) / len(perplexity_scores)}")

###Графички, шоб красиво :)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_metrics(rouge_scores, bleu_scores, perplexity_scores):
    indices = np.arange(len(rouge_scores))

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 3, 1)
    plt.plot(indices, rouge_scores, marker='o', linestyle='-', label='ROUGE-L')
    plt.xlabel("Наблюдения")
    plt.ylabel("Метрика")
    plt.title("ROUGE-L метрика")
    plt.grid(True)

    plt.subplot(1, 3, 2)
    plt.plot(indices, bleu_scores, marker='s', linestyle='-', label='BLEU', color='orange')
    plt.xlabel("Наблюдения")
    plt.ylabel("Метрика")
    plt.title("BLEU метрика")
    plt.grid(True)


    plt.subplot(1, 3, 3)
    plt.plot(indices, perplexity_scores, marker='^', linestyle='-', label='Perplexity', color='red')
    plt.xlabel("Наблюдения")
    plt.ylabel("Метрика")
    plt.title("Perplexity метрика")
    plt.grid(True)


    plt.tight_layout()
    plt.show()

plot_metrics(rouge_scores, bleu_scores, perplexity_scores)
